In [1]:
import pymupdf4llm

my_cv_path = "/Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf"

# Load pdf file
llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data(my_cv_path)

Successfully imported LlamaIndex
Processing /Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf...
[                                        ] (0/1)========================================[========================================] (1/1)]
Processing /Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf...
[                                        ] (0/1)========================================[========================================] (1/1)]


In [2]:
from IPython.display import Markdown, display

import chromadb
from chromadb.utils.embedding_functions import create_langchain_embedding
from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

# Using Huggingface embeddings
# langchain_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Using Ollama embeddings
langchain_embeddings = OllamaEmbeddings(model="llama3.2")
ef = create_langchain_embedding(langchain_embeddings)
embed_model = LangchainEmbedding(langchain_embeddings)

# Keeping option to switch between Ephemeral / Persistent chroma client, if wanted
chroma_db_persist_path = "chroma-db-job-coverletter"
vector_db_collection_name = "job-cv-vector-db-collection"
# ephemeral_chroma_client = chromadb.EphemeralClient()
persistent_chroma_client = chromadb.PersistentClient(path=chroma_db_persist_path)
vector_db_client = persistent_chroma_client

chroma_collection = vector_db_client.get_or_create_collection(
    vector_db_collection_name, embedding_function=ef
)

vector_store_from_client = Chroma(
    client=vector_db_client,
    collection_name=vector_db_collection_name,
    embedding_function=ef,
)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    llama_docs,
    storage_context=storage_context,
    embed_model=embed_model,
)

In [3]:
from langchain_ollama import ChatOllama

llm_instance = ChatOllama(model="llama3.2", temperature=0)

# Query Data
query_engine = index.as_query_engine(llm=llm_instance, streaming=True)
response = query_engine.query("What is the document all about?")
display(Markdown(f"<b>{response}</b>"))

<b>The document appears to be a resume or CV for an individual named Sukumar Raghavan, highlighting his professional experience, skills, and education in the field of machine learning engineering.</b>

In [5]:
from IPython.display import Markdown, display

# Use local Ollama model to generate the cover letter.
from chromadb.utils.embedding_functions import create_langchain_embedding
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

# Prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer just say "I don't know", don't try to make up an answer.
Context: {context}

Question: {question}

Answer as a jobseeker:"""

question = "Write a cover letter for applying for the job role as described in the context matching the job description exactly in the additional context."
llm = ChatOllama(model="llama3.2", temperature=0)
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

chroma_db_persist_path = "chroma-db-job-coverletter"
vector_db_collection_name = "job-cv-vector-db-collection"
lc_embeddings = OllamaEmbeddings(model="llama3.2")
# lc_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
ef2 = create_langchain_embedding(lc_embeddings)

vector_db = Chroma(
    collection_name=vector_db_collection_name,
    persist_directory=chroma_db_persist_path,
    embedding_function=ef2,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    chain_type="stuff", # other techniques to experiment: "map_reduce", "refine", "map_rerank"
)
retrieved_docs = vector_db.similarity_search("professional summary")
retrieved_docs += vector_db.similarity_search("work experience")
retrieved_context = "\n".join([doc.page_content for doc in retrieved_docs])

add_job_description_context = """
"""

context = f"{retrieved_context}\n\nAdditional Context:\n{add_job_description_context}"

result = qa_chain.invoke({"query": question, "context": context})

display(Markdown(f"<b>{result["result"]}</b>"))

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


<b>Dear Hiring Manager,

I am writing to express my enthusiastic interest in the Machine Learning Engineer position at your esteemed organization. As a seasoned machine learning engineer with over 6 years of experience in AI, I am confident that my skills and expertise align perfectly with the requirements of this role.

With a strong foundation in computer vision, NLP, CNN, LSTM, LLMs, vector databases, object detection, tracking, recommender systems, AutoML, model explainability, and distributed systems, I possess the technical expertise to develop cutting-edge AI solutions that address real-world problems. My experience with PyTorch, TensorFlow, Scikit-learn, Numpy, Pandas, Shap, XGBoost, OpenCV, Transformers, Langchain, LlamaIndex, ChromaDB, and other relevant technologies will enable me to make a significant impact in this role.

As a seasoned machine learning engineer, I have led the development of ContagionNET, significantly enhancing COVID-19 detection through innovative use of computer vision and deep learning. My experience with SHAP, model explainability, and trust has allowed me to develop visual AI for image classification that enhances user understanding and visualization of model performance.

I am particularly drawn to this role because of the opportunity to explore the potential of LLMs, Langchain, and vector databases to drive future innovation. I am eager to leverage my expertise in distributed systems, server architecture, and multithreading to develop scalable and efficient AI solutions that meet the organization's needs.

In addition to my technical skills, I possess excellent communication and collaboration skills, which have been essential in leading distributed teams across the globe. My experience as a cofounder and senior software architect has also honed my ability to establish roadmaps, create prototypes, and demo products to investors.

I am excited about the opportunity to join your team and contribute my expertise to develop cutting-edge AI solutions that drive business value. Thank you for considering my application. I look forward to discussing this opportunity further.

Sincerely,
Raghavan</b>